In [7]:
import tensorflow as tf
import numpy as np

import cv2
import os

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [8]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Model Load & Preprocessing

In [9]:
model = MyModel()
model.load_weights('./checkpoints/cifa10_checkpoint')

In [10]:
img = cv2.imread('./test_images/405.jpeg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img.reshape(-1,3).T.reshape(-1)
img = tf.convert_to_tensor([img], np.uint8)
img = tf.reshape(img, (-1, 3, 32, 32))
img = tf.transpose(img, (0, 2, 3, 1))
img = tf.cast(img, tf.float32)
img = img / 255

In [11]:
img.shape

TensorShape([1, 32, 32, 3])

# Model Run

In [12]:
predict = model(img)

In [13]:
print(np.argmax(predict))

4


# Modal Save

In [ ]:
model._set_inputs(img) 

In [18]:
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',   # Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0'    # If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})  

model.save("s3://model/cifar10/1")

INFO:tensorflow:Assets written to: s3://model/cifa10/1/assets


In [17]:
model.save("./test")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./test/assets
